In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

In [75]:
def get_data(pickall=True):
    
    data = pd.DataFrame()
    leagues = [file.lower() for file in listdir('./../test_data')]
    for league in leagues:
        files = [file.lower() for file in listdir(f'./../test_data/{league}')]
        print(files)

In [80]:
    data = pd.DataFrame()
    leagues = [file.lower() for file in listdir('./../test_data')]
    print(leagues)
    for league in leagues:
        files = [file.lower() for file in listdir(f'./../test_data/{league}')]
        print(league)

    

['eredivisie', 'turkey', 'greece']
eredivisie
turkey
greece


In [ ]:
    for league in leagues:
        for file in files:
            if file.startswith(league):
                df = pd.read_csv(f'./../test_data/{league}/'+file)
                data = pd.concat([data, df])

In [73]:
data = get_data()

In [74]:
data

""


In [55]:
data = pd.DataFrame()
leagues = [file.lower() for file in listdir('./../test_data')]
files = []
for league in leagues:
    files.append([file.lower() for file in listdir(f'./../test_data/{league}')])
for file in files:
    print(file)
    for league in leagues:
        if league in file:
            print(league)
            df = pd.read_csv(f'./../test_data/{league}/'+file)
            data = pd.concat([data, df])

[['eredivisie_2021_2022.csv']]
[['eredivisie_2021_2022.csv'], ['turkey_2021_2022.csv']]
[['eredivisie_2021_2022.csv'], ['turkey_2021_2022.csv'], ['greece_2021_2022.csv']]
['eredivisie_2021_2022.csv']
['turkey_2021_2022.csv']
['greece_2021_2022.csv']


In [39]:
data

""


In [81]:
def get_data(league1, league2=None, league3=None, league4=None, League5=None):
    
    files = [file for file in listdir(f'./../raw_data/{league1}')]
    data = pd.DataFrame()
    
    #For the case we have 2 leagues to concatenate:
    if league2:
        files1 = [file for file in listdir(f'./../raw_data/{league1}')]
        files2 = [file for file in listdir(f'./../raw_data/{league2}')]
        for file1 in files1:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file1)
            data = pd.concat([data, df])
        for file2 in files2:
            df = pd.read_csv(f'./../raw_data/{league2}/'+file2)
            data = pd.concat([data, df])
        return data
    
    #For the case we have 3 leagues to concatenate:
    if league2 and league3:
        files1 = [file for file in listdir(f'./../raw_data/{league1}')]
        files2 = [file for file in listdir(f'./../raw_data/{league2}')]
        files3 = [file for file in listdir(f'./../raw_data/{league3}')]
        
        for file1 in files1:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file1)
            data = pd.concat([data, df])
        for file2 in files2:
            df = pd.read_csv(f'./../raw_data/{league2}/'+file2)
            data = pd.concat([data, df])
        for file3 in files3:
            df = pd.read_csv(f'./../raw_data/{league3}/'+file3)
            data = pd.concat([data, df])
        return data
    
    #For the case we have 4 leagues to concatenate:
    if league2 and league3 and league4:
        files1 = [file for file in listdir(f'./../raw_data/{league1}')]
        files2 = [file for file in listdir(f'./../raw_data/{league2}')]
        files3 = [file for file in listdir(f'./../raw_data/{league3}')]
        files4 = [file for file in listdir(f'./../raw_data/{league4}')]
        
        for file1 in files1:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file1)
            data = pd.concat([data, df])
        for file2 in files2:
            df = pd.read_csv(f'./../raw_data/{league2}/'+file2)
            data = pd.concat([data, df])
        for file3 in files3:
            df = pd.read_csv(f'./../raw_data/{league3}/'+file3)
            data = pd.concat([data, df])
        for file4 in files4:
            df = pd.read_csv(f'./../raw_data/{league4}/'+file4)
            data = pd.concat([data, df])

        return data
    
    #For the case we have 5 leagues to concatenate:
    if league2 and league3 and league4 and league5:
        files1 = [file for file in listdir(f'./../raw_data/{league1}')]
        files2 = [file for file in listdir(f'./../raw_data/{league2}')]
        files3 = [file for file in listdir(f'./../raw_data/{league3}')]
        files4 = [file for file in listdir(f'./../raw_data/{league4}')]
        files5 = [file for file in listdir(f'./../raw_data/{league5}')]
        
        for file1 in files1:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file1)
            data = pd.concat([data, df])
        for file2 in files2:
            df = pd.read_csv(f'./../raw_data/{league2}/'+file2)
            data = pd.concat([data, df])
        for file3 in files3:
            df = pd.read_csv(f'./../raw_data/{league3}/'+file3)
            data = pd.concat([data, df])
        for file4 in files4:
            df = pd.read_csv(f'./../raw_data/{league4}/'+file4)
            data = pd.concat([data, df])
        for file5 in files5:
            df = pd.read_csv(f'./../raw_data/{league5}/'+file5)
            data = pd.concat([data, df])

        return data

In [70]:
data = get_data('turkey', league2='eredivisie', league3 = 'greece')

In [71]:
data

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,T1,11/08/17,Buyuksehyr,Bursaspor,1.0,0.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T1,12/08/17,Akhisar Belediyespor,Sivasspor,1.0,0.0,H,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T1,12/08/17,Alanyaspor,Kasimpasa,1.0,3.0,A,1.0,2.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,T1,12/08/17,Genclerbirligi,Karabukspor,1.0,1.0,D,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T1,12/08/17,Goztep,Fenerbahce,2.0,2.0,D,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,N1,15/05/2019,Graafschap,Ajax,1.0,4.0,A,1.0,2.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,N1,15/05/2019,NAC Breda,Zwolle,0.0,0.0,D,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,N1,15/05/2019,PSV Eindhoven,Heracles,3.0,1.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,N1,15/05/2019,Utrecht,Heerenveen,3.0,1.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
